<a href="https://colab.research.google.com/github/don3tan/MLCodingChallenge/blob/master/ML_Challenge_SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
'''SVC(C=10, break_ties=False, cache_size=8192, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='poly',
    max_iter=-1, probability=False, random_state=42, shrinking=True, tol=0.001,
    verbose=False)
89.55
'''


"SVC(C=10, break_ties=False, cache_size=8192, class_weight=None, coef0=0.0,\n    decision_function_shape='ovr', degree=3, gamma='scale', kernel='poly',\n    max_iter=-1, probability=False, random_state=42, shrinking=True, tol=0.001,\n    verbose=False)\n89.55\n"

# ML Coding Challenge SVM Notebook - Group: Strangers
> Ning Natalie Lee, Wan Ting Lee, Don Kok Jun Tan, Martin Klapper


# 1. Preparations

> ## 1.1 Importing Libraries + Prepare Dataset Download


In [0]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler, PowerTransformer
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from skimage.feature import hog
from sklearn.metrics import accuracy_score, confusion_matrix

import matplotlib.pyplot as plt

> ## 1.2 Confusion Matrix Labels

In [0]:
labels_dict = {0: "T-shirt/top",
               1: "Trouser",
               2: "Pullover",
               3: "Dress",
               4: "Coat",
               5: "Sandal",
               6: "Shirt",
               7: "Sneaker",
               8: "Bag",
               9: "Ankle boot"}

# 2. Data Preparation

> ## 2.1 Dataset Download

In [0]:
!git clone https://github.com/zalandoresearch/fashion-mnist.git

Cloning into 'fashion-mnist'...
remote: Enumerating objects: 688, done.
remote: Total 688 (delta 0), reused 0 (delta 0), pack-reused 688
Receiving objects: 100% (688/688), 105.20 MiB | 43.46 MiB/s, done.
Resolving deltas: 100% (397/397), done.


In [0]:
%cd /content/fashion-mnist/utils/
import mnist_reader

/content/fashion-mnist/utils


In [0]:
%cd

/root


In [0]:
x_train, y_train = mnist_reader.load_mnist('/content/fashion-mnist/data/fashion', kind='train')
x_test, y_test = mnist_reader.load_mnist('/content/fashion-mnist/data/fashion', kind='t10k')

In [0]:
x_train = x_train.astype('int')
y_train = y_train.astype('int')

x_test = x_test.astype('int')
y_test = y_test.astype('int')

> ## 2.2 Normalizing

In [0]:
#x_train = StandardScaler().fit_transform(x_train)
#x_test = StandardScaler().fit_transform(x_test)

In [0]:
x_train = x_train / 255
x_test = x_test / 255

In [0]:
x_train[0]

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.     

# 3. Feature Extraction (HOG)

In [0]:
x_train_features = []

for i, image in enumerate(x_train):

  train_feature = hog(x_train[i].reshape(28,28), orientations = 12, pixels_per_cell = (4,4), cells_per_block = (1,1), visualize = False, transform_sqrt = False)
  x_train_features.append(train_feature)

  if i % 10000 == 0:
    print('[LOG] {} features of training image {} succesfully extracted.'.format(str(len(train_feature)), str(i).zfill(5)))

[LOG] 588 features of training image 00000 succesfully extracted.
[LOG] 588 features of training image 10000 succesfully extracted.
[LOG] 588 features of training image 20000 succesfully extracted.
[LOG] 588 features of training image 30000 succesfully extracted.
[LOG] 588 features of training image 40000 succesfully extracted.
[LOG] 588 features of training image 50000 succesfully extracted.


In [0]:
x_test_features = []
for i, image in enumerate(x_test):

  test_feature = hog(x_test[i].reshape(28,28), orientations = 12, pixels_per_cell = (4,4), cells_per_block = (1,1), visualize = False, transform_sqrt = False)
  x_test_features.append(test_feature)

  if i % 1000 == 0:
    print('[LOG] {} features of testing image {} succesfully extracted.'.format(str(len(test_feature)), str(i).zfill(5)))

[LOG] 588 features of testing image 00000 succesfully extracted.
[LOG] 588 features of testing image 01000 succesfully extracted.
[LOG] 588 features of testing image 02000 succesfully extracted.
[LOG] 588 features of testing image 03000 succesfully extracted.
[LOG] 588 features of testing image 04000 succesfully extracted.
[LOG] 588 features of testing image 05000 succesfully extracted.
[LOG] 588 features of testing image 06000 succesfully extracted.
[LOG] 588 features of testing image 07000 succesfully extracted.
[LOG] 588 features of testing image 08000 succesfully extracted.
[LOG] 588 features of testing image 09000 succesfully extracted.


In [0]:
x_test_features[0].reshape(28,28)

ValueError: ignored

# 4. Training the SVM Classifier


In [0]:
x_test_features = x_test_features[:2000]
x_train_features = x_train_features[:10000]

In [0]:
y_train = y_train[:10000]
y_test = y_test[:2000]

In [0]:
classifier = SVC(random_state = 123, cache_size = 8192)

In [0]:
parameters = {'kernel':('rbf', 'poly', 'sigmoid'), 'C':[1, 5, 8, 9, 10, 11, 12, 15], 'gamma':['auto', 'scale']}

clf = GridSearchCV(classifier, param_grid = parameters, verbose = 10, n_jobs = -1)
clf.fit(x_train_features, y_train)
print(clf.best_estimator_)
print(clf.best_params_)
print(clf.best_score_)

Fitting 5 folds for each of 48 candidates, totalling 240 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:   53.1s


In [0]:
#tested with PowerTransform!

In [0]:
#svm = SVC(kernel = 'rbf', C = 6.5, gamma = 'auto', random_state = 123, cache_size = 8192)

In [0]:
#svm = SVC(kernel = 'poly', C=10, gamma = 'scale', random_state = 42, cache_size = 8192)

In [0]:
#svm.fit(x_train_features, y_train)

In [0]:
#prediction = svm.predict(x_test_features)

In [0]:
#print(accuracy_score(y_test, prediction))

1. Gridsearch poly degrees!

2. gridsearch with and without PowerTransform

3. gridsearch different hog masks??
